In [21]:
!pip install transformers[torch] datasets scikit-learn sacrebleu pandas

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [23]:
print("--- Phase 2: Preparing Data ---")
file_path = 'deformalization_dataset.csv'  # Make sure this matches the uploaded filename
try:
    df = pd.read_csv(file_path)

    # Rename columns for our task
    df.rename(columns={'input': 'input_text', 'output': 'target_text'}, inplace=True)
    df.dropna(inplace=True)

    # Split the data
    train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
    train_df, val_df = train_test_split(train_val_df, test_size=(1/9), random_state=42)

    # Convert to Hugging Face Dataset objects
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    test_dataset = Dataset.from_pandas(test_df)

    raw_datasets = DatasetDict({
        'train': train_dataset,
        'validation': val_dataset,
        'test': test_dataset
    })
    print("Data successfully loaded and split:")
    print(raw_datasets)
except FileNotFoundError:
    print(f"ERROR: Make sure you have uploaded '{file_path}' to the Colab session!")
except Exception as e:
    print(f"An error occurred during data preparation: {e}")

--- Phase 2: Preparing Data ---
Data successfully loaded and split:
DatasetDict({
    train: Dataset({
        features: ['lean_syntax', 'natural_language', '__index_level_0__'],
        num_rows: 1080
    })
    validation: Dataset({
        features: ['lean_syntax', 'natural_language', '__index_level_0__'],
        num_rows: 135
    })
    test: Dataset({
        features: ['lean_syntax', 'natural_language', '__index_level_0__'],
        num_rows: 135
    })
})


In [30]:
print("\n--- Phase 3: Model Training ---")
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Preprocessing function to tokenize data
prefix = "translate formal math to english: "
def preprocess_function(examples):
    inputs = [prefix + str(doc) for doc in examples["lean_syntax"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    labels = tokenizer(text_target=[str(doc) for doc in examples["natural_language"]], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results_deformalization",
    # MODIFIED LINE: Changed 'evaluation_strategy' to 'eval_strategy' for compatibility
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    predict_with_generate=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()
print("Training complete.")


--- Phase 3: Model Training ---


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

...

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

/tmp/ipython-input-750418915.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,No log,1.674510
2,No log,0.996703
3,No log,0.686231
4,No log,0.534162
5,No log,0.419679
6,No log,0.345061
7,No log,0.296081
8,1.110500,0.262371
9,1.110500,0.237468
10,1.110500,0.216675


Training complete.


In [25]:
!pip install evaluate

In [31]:
print("\n--- Phase 4: Evaluating Results ---")
import torch
from torch.utils.data import DataLoader
import numpy as np
import evaluate
from tqdm.auto import tqdm

def generate_predictions(model, tokenizer, dataset, collator):
    model.eval()
    model.to("cuda" if torch.cuda.is_available() else "cpu")

    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    dataloader = DataLoader(dataset, batch_size=32, collate_fn=collator)

    all_preds = []

    for batch in tqdm(dataloader, desc="Generating predictions"):
        input_ids = batch["input_ids"].to(model.device)
        attention_mask = batch["attention_mask"].to(model.device)

        generated_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=128)

        preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        all_preds.extend(preds)

    return all_preds

decoded_preds = generate_predictions(model, tokenizer, tokenized_datasets["test"], data_collator)

# MODIFIED LINE: Get ground truth labels from the original test_df using the correct column name
decoded_labels = test_df["natural_language"].tolist()

# Calculate BLEU score
bleu_metric = evaluate.load("sacrebleu")
result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
print(f"\nFINAL BLEU SCORE: {result['score']:.2f}")

# Display some example predictions
print("\n--- Example Predictions ---")
for i in range(min(5, len(test_df))):
    # MODIFIED LINE: Use the correct original column names for the final printout
    print(f"Formal Input:  {test_df.iloc[i]['lean_syntax']}")
    print(f"Ground Truth:  {test_df.iloc[i]['natural_language']}")
    print(f"Model Output:  {decoded_preds[i]}")
    print("-" * 20)


--- Phase 4: Evaluating Results ---


Generating predictions:   0%|          | 0/5 [00:00<?, ?it/s]


FINAL BLEU SCORE: 57.26

--- Example Predictions ---
Formal Input:  n ∈ T \ X
Ground Truth:  n is in T but not in X
Model Output:  n is in T but not in X
--------------------
Formal Input:  {x : ℝ | 0 < x ∧ x < 1}
Ground Truth:  The set of all real numbers between 0 and 1
Model Output:  The set of all real numbers between 0 and 1
--------------------
Formal Input:  ∀ x ∈ T, P x
Ground Truth:  For all x in T, property P holds
Model Output:  The empty set has no elements
--------------------
Formal Input:  𝒫(Y) = {S | S ⊆ Y}
Ground Truth:  The power set of Y is the set of all subsets of Y
Model Output:  The set of all subsets of Y
--------------------
Formal Input:  ∃ S ⊆ Y, S ≠ ∅
Ground Truth:  There exists a subset S of Y that is non-empty
Model Output:  The intersection of sets S and Y is the intersection of sets S and 
--------------------
